In [10]:
import pandas as pd
from imblearn.over_sampling import SMOTE
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
import statistics
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.simplefilter('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
from pandas import DataFrame
import statistics 
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from scipy import interp
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

    

In [30]:
with open("Result_Folder/RF_AllAmp_time.csv",'w') as resultfile:
    header='user_no,accu,happy_accu,sad_accu,stress_accu,relax_accu\n'
    resultfile.write(header)
    for root, dirs, files in os.walk("SMOTE_folder/Allamp_timeDomain/"):
        #files.remove(".~lock.user_1_v4.csv#")
        for filename in files:
            url="SMOTE_folder/Allamp_timeDomain/"+filename
            print(url)
            dataset = pd.read_csv(url,header=None)
            array = dataset.values
            col_no=array.shape[1]-1
            X_set=array[:,0:col_no]
            #X_set=array[:,0:2]
            #binarize the label
            Y_set=label_binarize(array[:,col_no],classes=['Happy','Sad','Stressed','Relaxed'])
            #Y_set=label_binarize(array[:,col_no],classes=[2.0,-2.0,1.0,0.0])
            n_classes=Y_set.shape[1]
            #print(Y_set)
            kfold = KFold(10, True, 1)
            cv = StratifiedKFold(n_splits=6)
            data_array=np.concatenate((X_set,Y_set),axis=1)
            #print(data_array.shape)
            roc_auc_avg=[]
            happy_list=[]
            sad_list=[]
            stressed_list=[]
            relaxed_list=[]
            dummy_col=col_no+4
            for train, test in kfold.split(data_array):
                #print('train: %s, test: %s' % (len(data_array[train]), len(data_array[test])))
                train_set=data_array[train]
                test_set=data_array[test]
                X_train=train_set[:,0:col_no]
                Y_train=train_set[:,col_no:dummy_col]
                Y_train=Y_train.astype('int')
                X_test=test_set[:,0:col_no]
                Y_test=test_set[:,col_no:dummy_col]
                Y_test=Y_test.astype('int')
                #print(Y_test.shape)
                fpr = dict()
                tpr = dict()
                roc_auc = dict()
                f_score=dict()
                #Learn to predict each class against the other
                clf=OneVsRestClassifier(RandomForestClassifier(n_estimators=100))
                #clf=OneVsRestClassifier(LogisticRegression(solver='liblinear', multi_class='ovr'))
                #clf=OneVsRestClassifier(KNeighborsClassifier())
                #clf=LogisticRegression(solver='liblinear', multi_class='ovr')
                Y_prob=clf.fit(X_train,Y_train).predict_proba(X_test)
                #Y_pred=clf.fit(X_train,Y_train).predict(X_test)
                #print(Y_pred)
                #compute AUCROC for each class
                for i in range(n_classes):
                    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], Y_prob[:, i])
                    roc_auc[i] = auc(fpr[i], tpr[i]) 
                    
                    
                    if(math.isnan(roc_auc[i])):
                        continue
                    elif(i==0):
                        happy_list.append(roc_auc[i])
                        
                    elif(i==1):
                        sad_list.append(roc_auc[i])
                        
                    elif(i==2):
                        stressed_list.append(roc_auc[i])
                        
                    else:
                        relaxed_list.append(roc_auc[i])
                        
                #print(roc_auc) 
                try:    
                    weighted_roc_auc_ovr = roc_auc_score(Y_test, Y_prob,multi_class="ovr",average="weighted")
                    #print(weighted_roc_auc_ovr)
                    roc_auc_avg.append(weighted_roc_auc_ovr)
                except ValueError:
                    pass

            if(len(roc_auc_avg)!=0):    
                 print("roc_auc_avg=",statistics.mean(roc_auc_avg))   
            #for auc-roc    
            happy_score,sad_score,stress_score,relax_score=0,0,0,0
            if(len(happy_list)!=0):
                happy_score=statistics.mean(happy_list)
                print("happy score=",happy_score)    
            if(len(sad_list)!=0):
                sad_score=statistics.mean(sad_list)
                print("sad_score=",sad_score)    
            if(len(stressed_list)!=0):
                stress_score=statistics.mean(stressed_list)
                print("stressed_score=",stress_score)    
            if(len(relaxed_list)!=0):    
                relax_score=statistics.mean(relaxed_list)
                print("relax_score=",relax_score)  
    
            roc_auc_weight=0

            for i in range(n_classes):
                no_entry=sum(Y_set[:,i]==1)
                if(i==0):
                    happy_entry=no_entry
                elif(i==1):
                     sad_entry=no_entry
                elif(i==2):
                    stress_entry=no_entry
                else:
                     relax_entry=no_entry
            roc_auc_weight=((happy_entry*happy_score)+(sad_entry*sad_score)+(stress_entry*stress_score)+(relax_score*relax_entry))/X_set.shape[0]    
            print("roc_auc_weight=",roc_auc_weight)  
            line=filename+','+str(roc_auc_weight)+','+str(happy_score)+','+str(sad_score)+','+str(stress_score)+','+str(relax_score)+'\n'
            resultfile.write(line)

SMOTE_folder/Allamp_timeDomain/U3_LIHF_itd_file.csv
happy score= 0.7
sad_score= 0.7
stressed_score= 0.84375
relax_score= 0.6166666666666667
roc_auc_weight= 0.6435763888888889
SMOTE_folder/Allamp_timeDomain/U4_LIHF_itd_file.csv
roc_auc_avg= 0.6306182706182706
happy score= 0.9571338383838384
sad_score= 0.8532467532467533
stressed_score= 0.5350008016674683
relax_score= 0.6345039682539683
roc_auc_weight= 0.6686240279005137
SMOTE_folder/Allamp_timeDomain/U10_LIHF_itd_file.csv
roc_auc_avg= 0.5203703703703704
happy score= 0.765625
sad_score= 0.4765432098765432
stressed_score= 0.5
relax_score= 0.6044973544973545
roc_auc_weight= 0.5753466098373506
SMOTE_folder/Allamp_timeDomain/U20_LIHF_itd_file.csv
roc_auc_avg= 0.6425242370328584
happy score= 0.5926221163096164
sad_score= 0.6179768223891245
stressed_score= 0.6575514069264069
relax_score= 0.8080533769063181
roc_auc_weight= 0.6507807054686124
SMOTE_folder/Allamp_timeDomain/U13_LIHF_itd_file.csv
roc_auc_avg= 0.5633928571428571
happy score= 0.5166